<a href="https://colab.research.google.com/github/salemsama/nn-zero-to-hero/blob/main/makemore1/makemore_part1_E04_trigram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful.
# Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

# result:
# used xs as index for w
# and ideed the 1-hot felt wasteful took a lot of time, now the code is much much faster


import torch

words = open("names.txt", 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
words[0]

'emma'

In [2]:
# this is trigram and exercise on bigram
# nerualnetwork version
xs = []
ys = []

for w in words:
  #we add 2 dots because we need a context (learned this idea from the video after this aka makemore 2)
  chs = ['.','.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    xs.append(stoi[ch1]*27 + stoi[ch2])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()

print(f"xs: {xs} \nys: {ys}")

#P = (N+1).float()
#P /= P.sum(1, keepdims=True)
#g = torch.Generator().manual_seed(2147483647)

xs: tensor([  0,   5, 148,  ..., 727, 701, 726]) 
ys: tensor([ 5, 13, 13,  ..., 26, 24,  0])


In [3]:
# encodeing xs
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes = 729).float() # 729 = 27 * 27 because we have 2 chars

In [4]:
xenc[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [5]:
xenc.shape # n * 729 where n is number of input aka

torch.Size([228146, 729])

In [6]:
# initilize weights
gen = torch.Generator().manual_seed(2147483647)
w = torch.randn((729,27), generator=gen, requires_grad=True)

In [15]:
for k in range(1000):
  # forward pass
  logits = w[xs]     #xenc @ w
  counts = logits.exp()
  probs  = counts / counts.sum(1, keepdim=True)

  loss = -probs[torch.arange(num), ys].log().mean()
  # backward pass
  w.grad = None # zero grad
  loss.backward()

  #update
  w.data += -10 * w.grad
  print(loss.item())


2.285640001296997
2.2856035232543945
2.285567045211792
2.2855303287506104
2.2854936122894287
2.2854573726654053
2.2854208946228027
2.285384178161621
2.2853479385375977
2.285311460494995
2.2852749824523926
2.285238742828369
2.2852022647857666
2.285166025161743
2.2851297855377197
2.2850935459136963
2.2850570678710938
2.2850210666656494
2.284985065460205
2.2849488258361816
2.284912586212158
2.284876585006714
2.2848405838012695
2.284804582595825
2.2847683429718018
2.2847325801849365
2.284696340560913
2.2846603393554688
2.2846245765686035
2.2845888137817383
2.284553050994873
2.2845170497894287
2.2844815254211426
2.2844455242156982
2.284409999847412
2.284374237060547
2.2843387126922607
2.2843029499053955
2.2842671871185303
2.284231424331665
2.284196138381958
2.284160614013672
2.2841250896453857
2.2840895652770996
2.2840540409088135
2.2840187549591064
2.2839832305908203
2.2839481830596924
2.283912420272827
2.28387713432312
2.283841848373413
2.283806800842285
2.283771514892578
2.28373622894287

In [8]:
loss.item()

2.9721181392669678

In [14]:
#sampling
for i in range(5):
  out = []
  ix1 = 0
  ix2 = 0
  while True:
    xenc = [ix1*27 + ix2]
    xenc = F.one_hot(torch.tensor(xenc), num_classes=729).float()
    logits = xenc @ w
    counts = logits.exp()
    p = counts / counts.sum(1, keepdims=True)

    ix1 = ix2
    ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=gen).item()
    out.append(itos[ix2])
    if ix2 == 0:
      break
  print(''.join(out))


kellovandomayah.
eah.
hwapcwwafyhcaizel.
amill.
delon.
